In [ ]:
import numpy as np
import spacy
import torch
from transformers import pipeline

from selfcheckgpt.modeling_selfcheck import SelfCheckLLMPrompt

In [2]:
torch.manual_seed(28)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# We use Phi-2 2.7B SLM for inferencing
pipe = pipeline("text-generation", model="microsoft/phi-2", device_map="auto")

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.73s/it]


In [5]:
prompt = """
Give me the professional journey of Ashish Vaswani in detail.
Answer:
"""

In [6]:
# As per the original paper the response is generated with greedy decoding
Response = pipe(prompt, do_sample=False, max_new_tokens=128, return_full_text=False)
Response

[{'generated_text': 'Ashish Vaswani started his career as a software engineer in a small startup company. He worked hard and gained experience in various projects, which helped him climb up the corporate ladder. He then joined a multinational company as a project manager and was responsible for managing a team of developers. His leadership skills and dedication towards his work were recognized, and he was promoted to the position of a senior project manager.\n\nAfter a few years, Ashish decided to pursue his passion for entrepreneurship and started his own software development company. He faced many challenges in the initial stages, but with his determination and hard work, he was able to establish a'}]

In [7]:
# The samples are generated for the same prompt with temperature as 1.
N = 20
Samples = pipe(
    [prompt] * N,
    temperature=1.0,
    do_sample=True,
    max_new_tokens=128,
    return_full_text=False,
)
print(Samples[0])

[{'generated_text': "Ashish Vaswani has always been passionate about the Indian film industry, and he pursued his interest by working as a producer and writer. He started his career by serving his family's business and then moved on to other jobs before finally entering the film industry.\nAs a producer and screenwriter, Ashish has a diverse background, having worked on multiple projects such as music videos, TV commercials, short films, documentaries, and web series. He gained recognition for his work and has worked extensively in the Telugu film industry since 2017.\nAshish's journey in the film industry has been a learning curve, and he has been involved"}]


In [8]:
Response = Response[0]["generated_text"]
Samples = [sample[0]["generated_text"] for sample in Samples]

In [ ]:
# Mistral 7B became a gated repository so huggingface login is required to access it.
from huggingface_hub import login

HUGGINGFACE_TOKEN = "..."
login(token=HUGGINGFACE_TOKEN)

In [11]:
# We use Mistral 7B LLM to detect whether the response generated with Phi-2 LM is hallucinated or not using LLM Promting technique.
llm_model = "mistralai/Mistral-7B-Instruct-v0.2"
selfcheck_prompt = SelfCheckLLMPrompt(llm_model, device)

# Option2: API access (currently only support client_type="openai")
# from selfcheckgpt.modeling_selfcheck_apiprompt import SelfCheckAPIPrompt
# selfcheck_prompt = SelfCheckAPIPrompt(client_type="openai", model="gpt-3.5-turbo")

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.99s/it]


SelfCheck-LLMPrompt (mistralai/Mistral-7B-Instruct-v0.2) initialized to device cuda


In [12]:
nlp = spacy.load("en_core_web_sm")
sentences = [
    sent.text.strip() for sent in nlp(Response).sents
]  # spacy sentence tokenization
print(sentences)

sent_scores_prompt = selfcheck_prompt.predict(
    sentences=sentences,  # list of sentences
    sampled_passages=Samples,  # list of sampled passages
    verbose=True,  # whether to show a progress bar
)

print(sent_scores_prompt)
print("Hallucination Score:", np.mean(sent_scores_prompt))

['Ashish Vaswani started his career as a software engineer in a small startup company.', 'He worked hard and gained experience in various projects, which helped him climb up the corporate ladder.', 'He then joined a multinational company as a project manager and was responsible for managing a team of developers.', 'His leadership skills and dedication towards his work were recognized, and he was promoted to the position of a senior project manager.', 'After a few years, Ashish decided to pursue his passion for entrepreneurship and started his own software development company.', 'He faced many challenges in the initial stages, but with his determination and hard work, he was able to establish a']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:58<00:00,  9.79s/it]

[1.   0.3  0.75 0.6  0.65 0.05]
Hallucination Score: 0.5583333333333332
